# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

#Configure google mapes with API 
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
# Retrieve 
city_weather_file = "../cityweather.csv"

In [13]:
#Read CSV 
city_weather = pd.read_csv('cityweather.csv')

In [14]:
weather_df = city_weather.drop(columns=["Unnamed: 0"])
weather_df


,Name,Lat,Lon,Temp,Humidity,WindSpeed,Clouds
0,Rikitea,-23.1203,-134.9692,299.28,83,7.55,79
1,Punta Arenas,-53.1500,-70.9167,277.09,70,6.17,75
2,Hilo,19.7297,-155.0900,297.53,57,4.63,75
3,Avarua,-21.2078,-159.7750,300.15,65,3.09,75
4,Mataura,-46.1927,168.8643,287.59,81,3.58,100
...,...,...,...,...,...,...,...
550,Kostomuksha,64.5710,30.5767,271.59,84,4.18,100
551,Arlit,18.7369,7.3853,303.84,5,3.75,0
552,Muisne,0.6000,-80.0333,299.15,89,1.54,75
553,Zharkent,44.1628,80.0000,287.60,39,2.36,74


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
#Configure google mapes with API 
gmaps.configure(api_key=g_key)

In [16]:
# Use latitude and longitude as locations
locations = weather_df[["Lat", "Lon"]]

#Add humidity layer to heatmap 
humidity = weather_df["Humidity"]

In [17]:
#Plotting heat map layer to map 
fig = gmaps.figure(center =  [0,0], zoom_level=2)

#Make heat map layer
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                    dissipating=False, max_intensity=5,
                                    point_radius=1)

fig.add_layer(heatmap_layer)

#Display
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
new_weather_df = pd.DataFrame(weather_df, columns =["Name", "Temp", "WindSpeed", "Clouds"])

temperature = (new_weather_df ["Temp"] <=80) & (new_weather_df["Temp"]>70)

wind_speed = new_weather_df["WindSpeed"]<10

cloudiness = new_weather_df["Clouds"]== 0

new_weather_df[temperature & wind_speed & cloudiness]
new_weather_df

,Name,Temp,WindSpeed,Clouds
0,Rikitea,299.28,7.55,79
1,Punta Arenas,277.09,6.17,75
2,Hilo,297.53,4.63,75
3,Avarua,300.15,3.09,75
4,Mataura,287.59,3.58,100
...,...,...,...,...
550,Kostomuksha,271.59,4.18,100
551,Arlit,303.84,3.75,0
552,Muisne,299.15,1.54,75
553,Zharkent,287.60,2.36,74


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
hotel_df = pd.DataFrame(weather_df, columns=["Name", "Lat", "Lon"])

hotel_df["Hotel Name"] = " "
hotel_df

,Name,Lat,Lon,Hotel Name
0,Rikitea,-23.1203,-134.9692,
1,Punta Arenas,-53.1500,-70.9167,
2,Hilo,19.7297,-155.0900,
3,Avarua,-21.2078,-159.7750,
4,Mataura,-46.1927,168.8643,
...,...,...,...,...
550,Kostomuksha,64.5710,30.5767,
551,Arlit,18.7369,7.3853,
552,Muisne,0.6000,-80.0333,
553,Zharkent,44.1628,80.0000,


In [33]:
#base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "type" : "hotel",
    "keyword" : "hotel",
    "radius" : 5000,
    "key" : g_key
}

In [39]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lon"]
    city_name = row["Name"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try: 
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"]= results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("------------------------")
        
       
    print("-----------------------Ending Search")

Retrieving Results for Index 0: Rikitea.
Could not find result
------------------------
-----------------------Ending Search
Retrieving Results for Index 1: Punta Arenas.
Closest hotel in Punta Arenas is Hotel Dreams Del Estrecho.
-----------------------Ending Search
Retrieving Results for Index 2: Hilo.
Closest hotel in Hilo is Hilo Hawaiian Hotel.
-----------------------Ending Search
Retrieving Results for Index 3: Avarua.
Closest hotel in Avarua is The Islander Hotel.
-----------------------Ending Search
Retrieving Results for Index 4: Mataura.
Could not find result
------------------------
-----------------------Ending Search
Retrieving Results for Index 5: Ponta do Sol.
Closest hotel in Ponta do Sol is Estalagem Da Ponta Do Sol.
-----------------------Ending Search
Retrieving Results for Index 6: Vaini.
Closest hotel in Vaini is Paradise First Hotel.
-----------------------Ending Search
Retrieving Results for Index 7: Ushuaia.
Closest hotel in Ushuaia is Los Cauquenes Resort & Spa

Closest hotel in Tasiilaq is Hotel Angmagssalik.
-----------------------Ending Search
Retrieving Results for Index 64: Severo-Kuril'sk.
Closest hotel in Severo-Kuril'sk is Kak Doma.
-----------------------Ending Search
Retrieving Results for Index 65: Whitehorse.
Closest hotel in Whitehorse is Best Western Gold Rush Inn.
-----------------------Ending Search
Retrieving Results for Index 66: São Félix do Xingu.
Closest hotel in São Félix do Xingu is Hotel Vista Rara.
-----------------------Ending Search
Retrieving Results for Index 67: Faya.
Could not find result
------------------------
-----------------------Ending Search
Retrieving Results for Index 68: Kendari.
Closest hotel in Kendari is Claro Hotel Kendari.
-----------------------Ending Search
Retrieving Results for Index 69: Tarauacá.
Closest hotel in Tarauacá is Ouro Hotel.
-----------------------Ending Search
Retrieving Results for Index 70: Dudinka.
Closest hotel in Dudinka is Yenisei lights.
-----------------------Ending Searc

Closest hotel in São Gabriel is Hotel San Isidro.
-----------------------Ending Search
Retrieving Results for Index 126: George Town.
Closest hotel in George Town is Eastern & Oriental Hotel, Penang.
-----------------------Ending Search
Retrieving Results for Index 127: Yanam.
Closest hotel in Yanam is Hotel Sarvainn.
-----------------------Ending Search
Retrieving Results for Index 128: Lang Son.
Closest hotel in Lang Son is Mường Thanh Hotel Lạng Sơn.
-----------------------Ending Search
Retrieving Results for Index 129: Eureka.
Closest hotel in Eureka is Best Western Plus Humboldt Bay Inn.
-----------------------Ending Search
Retrieving Results for Index 130: Suntar.
Closest hotel in Suntar is Gostevoy Dom.
-----------------------Ending Search
Retrieving Results for Index 131: Hithadhoo.
Closest hotel in Hithadhoo is Pebbles Inn.
-----------------------Ending Search
Retrieving Results for Index 132: Banjar.
Closest hotel in Banjar is Padmasari Resort Hotel.
-----------------------En

KeyboardInterrupt: 

In [40]:
hotel_df.head()

,Name,Lat,Lon,Hotel Name
0,Rikitea,-23.1203,-134.9692,
1,Punta Arenas,-53.1500,-70.9167,Hotel Dreams Del Estrecho
2,Hilo,19.7297,-155.0900,Hilo Hawaiian Hotel
3,Avarua,-21.2078,-159.7750,The Islander Hotel
4,Mataura,-46.1927,168.8643,


In [42]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Name}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [43]:
# Add marker layer ontop of heat map
# Display figure
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))